# Session 2 Chamodi : Gait Analysis
### Time : 2025-05-31 21:44:45.469000 to 2025-05-31 21:45:25.529000
### Sensor ID : 601
### Steps : 30

## Fetching Raw Data from DynamoDB

In [ ]:
import sys
import os
from datetime import datetime

# Append app to sys.path to access modules like app.utils.dynamo
sys.path.append(os.path.abspath("../app"))

In [6]:
from utils.dynamo import fetch_session_data

In [9]:
sensor_id = 601
start_time = datetime.fromisoformat("2025-05-31 19:57:33.521000")
end_time = datetime.fromisoformat("2025-05-31 19:57:57.172000")

data = fetch_session_data(sensor_id, start_time, end_time)
print(f"📦 Retrieved {len(data)} records")


📦 Retrieved 0 records


In [4]:
data

[]

## Step 1 :  Preprocess and Sort

### Sort By Timestamp

In [10]:
import pandas as pd
from decimal import Decimal

# Convert to DataFrame
df = pd.json_normalize(data)

# Convert all Decimal values to float (optional but useful)
df = df.map(lambda x: float(x) if isinstance(x, Decimal) else x)

# Sort by timestamp
df = df.sort_values(by="timestamp").reset_index(drop=True)

# Show first few rows
df.head()


,gyro_cal,mag_cal,roll,type,FSR_12,FSR_11,FSR_10,FSR_16,FSR_15,FSR_1,...,device_id,yaw,gx,gy,gz,sys_cal,accel_cal,ax,ay,az
0,3.0,3.0,7.62,sensor_data,0.0,0.0,0.0,0.0,147.0,0.0,...,601,16.81,10.94,-46.44,26.87,3.0,3.0,2.35,-0.65,0.27
1,3.0,3.0,-2.56,sensor_data,0.0,0.0,0.0,0.0,147.0,0.0,...,601,29.37,-3.69,-13.00,-15.13,3.0,3.0,-0.55,-0.03,0.26
2,3.0,3.0,-5.87,sensor_data,0.0,0.0,0.0,0.0,148.0,0.0,...,601,6.44,-8.69,54.75,-46.31,3.0,3.0,-5.53,0.86,2.20
3,3.0,3.0,18.00,sensor_data,0.0,0.0,0.0,0.0,152.0,0.0,...,601,76.94,-45.81,-14.63,-128.75,3.0,3.0,3.04,2.05,-5.20
4,3.0,3.0,-5.25,sensor_data,0.0,0.0,0.0,0.0,139.0,0.0,...,601,39.88,-1.19,-27.87,-10.69,3.0,3.0,-0.24,-0.19,-0.13


### Check for Missing FSR Values

In [11]:
required_fsrs = [f"FSR_{i}" for i in range(1, 17)]
missing = [col for col in required_fsrs if col not in df.columns]
print("Missing FSR columns:", missing)

Missing FSR columns: []


### Clipping FSR values (0 to 4095)

In [12]:
for col in required_fsrs:
    df[col] = df[col].clip(0, 4095)


### Normalizing  Timestamps 

In [13]:
df["time_sec"] = df["timestamp"] - df["timestamp"].min()